<a href="https://colab.research.google.com/github/israinamdar493-ops/retinalprojectsizz4/blob/main/finalproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

# Go inside your Drive project folder
%cd /content/drive/MyDrive/retinalnewproject

# (Optional) check the structure
!ls -R | head -40   # shows first 40 lines of folder tree

# Install all dependencies
!pip install torch torchvision timm albumentations pandas scikit-learn xgboost opencv-python matplotlib

# >>> RUN YOUR SCRIPT (Phase-1 demo or real data) <<<
# Replace with the actual file you want to run, for example:
!python src/phase1_demo.py


Mounted at /content/drive
/content/drive/MyDrive/retinalnewproject
.:
datasets
ensemble
federated
outputs
preprocessing
reports
results
segmentation

./datasets:
APTOS
CHASEDB1
DRIVE
Messidor2
ODIR5K
RFMiD

./datasets/APTOS:
test.csv
test_images
train_1.csv
train_images
valid.csv
val_images

./datasets/APTOS/test_images:
test_images

./datasets/APTOS/test_images/test_images:
e4dcca36ceb4.png
e50b0174690d.png
e5197d77ec68.png
e529c5757d64.png
e594c19e2e1d.png
e5de79795c1d.png
e60e4edb3ca9.png
e6552b7432b3.png
e66855a5c583.png
e68746d426b2.png
python3: can't open file '/content/drive/MyDrive/retinalnewproject/src/phase1_demo.py': [Errno 2] No such file or directory


In [3]:
# Save this cell as a .py file inside your Drive
demo_code = """
import torch, torchvision
import pandas as pd
import os
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

print('✅ Demo running with Torch version:', torch.__version__)

# Define dataset paths - These should match the variables defined in the notebook
train_csv = "/content/drive/MyDrive/retinalnewproject/datasets/APTOS/train_1.csv"
train_dir = "/content/drive/MyDrive/retinalnewproject/datasets/APTOS/train_images"
val_csv = "/content/drive/MyDrive/retinalnewproject/datasets/APTOS/valid.csv"
val_dir = "/content/drive/MyDrive/retinalnewproject/datasets/APTOS/val_images"


class RetinalDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.df = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.df.iloc[idx, 0] + '.png')
        image = Image.open(img_name)
        label = self.df.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        return image, label

# Define a simple transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Create datasets and dataloaders
try:
    train_dataset = RetinalDataset(train_csv, train_dir, transform)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    print(f"✅ Loaded training dataset with {len(train_dataset)} images.")

    val_dataset = RetinalDataset(val_csv, val_dir, transform)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
    print(f"✅ Loaded validation dataset with {len(val_dataset)} images.")

except FileNotFoundError as e:
    print(f"❌ File not found error: {e}")
    print("Please ensure the dataset paths in the script are correct.")
    exit() # Exit the script if dataset loading fails

# Fake training loop
for epoch in range(1,4):
    print(f'Epoch {epoch}: training...done!')
print('✅ Finished demo training.')
"""
with open('/content/drive/MyDrive/retinalnewproject/phase1_demo.py', 'w') as f:
    f.write(demo_code)

print("✅ File created at /content/drive/MyDrive/retinalnewproject/phase1_demo.py")

✅ File created at /content/drive/MyDrive/retinalnewproject/phase1_demo.py


In [4]:
!python /content/drive/MyDrive/retinalnewproject/phase1_demo.py


✅ Demo running with Torch version: 2.8.0+cu126
Epoch 1: training...done!
Epoch 2: training...done!
Epoch 3: training...done!
✅ Finished demo training.


In [7]:
!python /content/drive/MyDrive/retinalnewproject/phase1_demo.py


Traceback (most recent call last):
  File "/content/drive/MyDrive/retinalnewproject/phase1_demo.py", line 58, in <module>
    train_dataset = RetinalDataset(train_csv, train_dir, transform)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/drive/MyDrive/retinalnewproject/phase1_demo.py", line 18, in __init__
    self.df = pd.read_csv(csv_file)
              ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parsers/readers.py", line 1026, in read_csv
    return _read(filepath_or_buffer, kwds)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parsers/readers.py", line 620, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pandas/io/parsers/readers.py", line 1620, in __init__
    self._engine = self._make_engine(f, self.engine)
                   ^^^^^^^^^^^

In [9]:
import os

base_path = "/content/drive/MyDrive/retinalnewproject/datasets/APTOS"

train_csv = os.path.join(base_path, "train_1.csv")
train_dir = os.path.join(base_path, "train_images")

val_csv = os.path.join(base_path, "valid.csv")
val_dir = os.path.join(base_path, "val_images")

In [12]:
base_path = "/content/drive/MyDrive/retinalnewproject/datasets/ODIR5K"


In [13]:
base_path = "/content/drive/MyDrive/retinalnewproject/datasets/RFMiD"


In [14]:
# phase1_demo.py
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

# -------------------------------
# 0. Select Dataset
# -------------------------------
# Change this to "APTOS", "ODIR5K", or "RFMiD"
DATASET_NAME = "APTOS"

base_path = f"/content/drive/MyDrive/retinalnewproject/datasets/{DATASET_NAME}"

train_csv = os.path.join(base_path, "train_1.csv")
train_dir = os.path.join(base_path, "train_images")

val_csv = os.path.join(base_path, "valid.csv")
val_dir = os.path.join(base_path, "val_images")

# -------------------------------
# 1. Dataset Class
# -------------------------------
class RetinalDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.df = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform

        # map labels to integers if not numeric
        if not np.issubdtype(self.df['label'].dtype, np.integer):
            self.df['label'] = pd.Categorical(self.df['label']).codes

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row['image'])
        image = Image.open(img_path).convert('RGB')
        label = torch.tensor(row['label'], dtype=torch.long)
        if self.transform:
            image = self.transform(image)
        return image, label

# -------------------------------
# 2. Transformations
# -------------------------------
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

# -------------------------------
# 3. Load Datasets
# -------------------------------
train_dataset = RetinalDataset(train_csv, train_dir, transform)
val_dataset   = RetinalDataset(val_csv, val_dir, transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=16, shuffle=False)

# -------------------------------
# 4. Model
# -------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(pd.unique(train_dataset.df['label']))
model = models.efficientnet_b0(weights=None, num_classes=num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# -------------------------------
# 5. Training Loop
# -------------------------------
epochs = 3  # Increase for longer training
for epoch in range(epochs):
    model.train()
    running_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs} | Train Loss: {running_loss/len(train_loader):.4f}")

# -------------------------------
# 6. Validation
# -------------------------------
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

acc = accuracy_score(all_labels, all_preds)
f1  = f1_score(all_labels, all_preds, average='weighted')
print(f"Validation Accuracy: {acc:.4f} | F1 Score: {f1:.4f}")

# -------------------------------
# 7. Save Model
# -------------------------------
results_path = "/content/drive/MyDrive/retinalnewproject/results"
os.makedirs(results_path, exist_ok=True)
torch.save(model.state_dict(), os.path.join(results_path, f"{DATASET_NAME}_efficientnet_phase1.pth"))
print(f"✅ Model saved to {results_path}/{DATASET_NAME}_efficientnet_phase1.pth")


KeyError: 'label'

In [15]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/retinalnewproject/datasets/APTOS/train_1.csv")
print(df.columns)


Index(['id_code', 'diagnosis'], dtype='object')


In [16]:
!python /content/drive/MyDrive/retinalnewproject/phase1_demo.py

Traceback (most recent call last):
  File "/content/drive/MyDrive/retinalnewproject/phase1_demo.py", line 89, in <module>
    for images, labels in train_loader:
                          ^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 734, in __next__
    data = self._next_data()
           ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 790, in _next_data
    data = self._dataset_fetcher.fetch(index)  # may raise StopIteration
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/content/drive/MyDrive/retinalnewproject/phase1_demo.py", line 47, in __getitem__
    image = Image.open(img_path).convert('RGB')
            ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages

In [17]:
# Original
train_dir = os.path.join(base_path, "train_images")
val_dir   = os.path.join(base_path, "val_images")

# Corrected (include the extra 'test_images' folder)
train_dir = os.path.join(base_path, "train_images", "test_images")
val_dir   = os.path.join(base_path, "val_images", "val_images")  # check val_images folder similarly


In [18]:
!ls /content/drive/MyDrive/retinalnewproject/datasets/APTOS/train_images/


train_images


In [25]:
!python /content/drive/MyDrive/retinalnewproject/phase1_demo.py


Train folder: /content/drive/MyDrive/retinalnewproject/datasets/APTOS/train_images/train_images
Val folder:   /content/drive/MyDrive/retinalnewproject/datasets/APTOS/val_images/val_images
Test folder:  /content/drive/MyDrive/retinalnewproject/datasets/APTOS/test_images/test_images
Traceback (most recent call last):
  File "/content/drive/MyDrive/retinalnewproject/phase1_demo.py", line 118, in <module>
    for images, labels in train_loader:
                          ^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 734, in __next__
    data = self._next_data()
           ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 790, in _next_data
    data = self._dataset_fetcher.fetch(index)  # may raise StopIteration
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] 

In [26]:
# phase1_demo.py
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

# -------------------------------
# 0. Dataset Paths
# -------------------------------
DATASET_NAME = "APTOS"
base_path = f"/content/drive/MyDrive/retinalnewproject/datasets/{DATASET_NAME}"

train_csv = os.path.join(base_path, "train_1.csv")
val_csv   = os.path.join(base_path, "valid.csv")
test_csv  = os.path.join(base_path, "test.csv")  # optional

# -------------------------------
# 1. Recursive folder detection
# -------------------------------
def get_image_folder(base_folder):
    """Recursively find first folder containing image files."""
    for root, dirs, files in os.walk(base_folder):
        img_files = [f for f in files if f.lower().endswith(('.png','.jpg','.jpeg'))]
        if len(img_files) > 0:
            return root
    raise ValueError(f"No image files found in {base_folder}")

train_dir = get_image_folder(os.path.join(base_path, "train_images"))
val_dir   = get_image_folder(os.path.join(base_path, "val_images"))
test_dir  = get_image_folder(os.path.join(base_path, "test_images"))

print(f"Train folder: {train_dir}")
print(f"Val folder:   {val_dir}")
print(f"Test folder:  {test_dir}")

# -------------------------------
# 2. Dataset Class
# -------------------------------
class RetinalDataset(Dataset):
    def __init__(self, csv_file, img_dir, label_col='diagnosis', transform=None):
        self.df = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        self.label_col = label_col

        self.df['id_code'] = self.df['id_code'].astype(str).str.strip()

        # Map labels to integers if not numeric
        if not np.issubdtype(self.df[self.label_col].dtype, np.integer):
            self.df[self.label_col] = pd.Categorical(self.df[self.label_col]).codes

        # Build mapping from id_code -> full image path
        self.id_to_path = {}
        for root, dirs, files in os.walk(self.img_dir):
            for f in files:
                name, ext = os.path.splitext(f)
                if name in self.df['id_code'].values and ext.lower() in ['.png','.jpg','.jpeg']:
                    self.id_to_path[name] = os.path.join(root, f)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        id_code = row['id_code']
        if id_code not in self.id_to_path:
            raise FileNotFoundError(f"Image file not found for id {id_code} in {self.img_dir}")
        img_path = self.id_to_path[id_code]

        image = Image.open(img_path).convert('RGB')
        label = torch.tensor(row[self.label_col], dtype=torch.long)
        if self.transform:
            image = self.transform(image)
        return image, label

# -------------------------------
# 3. Transformations
# -------------------------------
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

# -------------------------------
# 4. Load datasets
# -------------------------------
train_dataset = RetinalDataset(train_csv, train_dir, label_col='diagnosis', transform=transform)
val_dataset   = RetinalDataset(val_csv, val_dir, label_col='diagnosis', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=16, shuffle=False)

# -------------------------------
# 5. Model
# -------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(pd.unique(train_dataset.df['diagnosis']))
model = models.efficientnet_b0(weights=None, num_classes=num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# -------------------------------
# 6. Training loop
# -------------------------------
epochs = 3  # increase for better accuracy
for epoch in range(epochs):
    model.train()
    running_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs} | Train Loss: {running_loss/len(train_loader):.4f}")

# -------------------------------
# 7. Validation
# -------------------------------
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

acc = accuracy_score(all_labels, all_preds)
f1  = f1_score(all_labels, all_preds, average='weighted')
print(f"Validation Accuracy: {acc:.4f} | F1 Score: {f1:.4f}")

# -------------------------------
# 8. Save model
# -------------------------------
results_path = "/content/drive/MyDrive/retinalnewproject/results"
os.makedirs(results_path, exist_ok=True)
torch.save(model.state_dict(), os.path.join(results_path, f"{DATASET_NAME}_efficientnet_phase1.pth"))
print(f"✅ Model saved to {results_path}/{DATASET_NAME}_efficientnet_phase1.pth")


Train folder: /content/drive/MyDrive/retinalnewproject/datasets/APTOS/train_images/train_images
Val folder:   /content/drive/MyDrive/retinalnewproject/datasets/APTOS/val_images/val_images
Test folder:  /content/drive/MyDrive/retinalnewproject/datasets/APTOS/test_images/test_images


FileNotFoundError: Image file not found for id 916ec976ff30 in /content/drive/MyDrive/retinalnewproject/datasets/APTOS/train_images/train_images

In [28]:
from google.colab import drive
drive.mount('/content/drive')

!python /content/drive/MyDrive/retinalnewproject/phase1_demo.py


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Train folder: /content/drive/MyDrive/retinalnewproject/datasets/APTOS/train_images/train_images
Val folder:   /content/drive/MyDrive/retinalnewproject/datasets/APTOS/val_images/val_images
Test folder:  /content/drive/MyDrive/retinalnewproject/datasets/APTOS/test_images/test_images
Using 967 images from /content/drive/MyDrive/retinalnewproject/datasets/APTOS/train_images/train_images
Using 112 images from /content/drive/MyDrive/retinalnewproject/datasets/APTOS/val_images/val_images
Epoch 1/3 | Train Loss: 3.2950
Epoch 2/3 | Train Loss: 1.2566
Epoch 3/3 | Train Loss: 0.7312
Validation Accuracy: 0.9018 | F1 Score: 0.8666
✅ Model saved to /content/drive/MyDrive/retinalnewproject/results/APTOS_efficientnet_phase1.pth


In [29]:
# Install Gradio if not installed
!pip install gradio --quiet

import torch
from torchvision import models, transforms
from PIL import Image
import gradio as gr
import matplotlib.pyplot as plt
import numpy as np

# -------------------
# Settings
# -------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
num_classes = 5  # change to your number of disease classes
model_path = "/content/drive/MyDrive/retinalnewproject/results/APTOS_efficientnet_phase1.pth"

# -------------------
# Image preprocessing
# -------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# -------------------
# Load model
# -------------------
model = models.efficientnet_b0(weights=None, num_classes=num_classes)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()
model.to(device)

# -------------------
# Grad-CAM function
# -------------------
def grad_cam(input_image, class_idx=None):
    # Hook for gradients
    gradients = []
    def save_gradients(module, grad_input, grad_output):
        gradients.append(grad_output[0])
    target_layer = model.features[-1]
    target_layer.register_backward_hook(save_gradients)

    # Prepare image
    input_tensor = transform(input_image).unsqueeze(0).to(device)
    input_tensor.requires_grad = True

    # Forward pass
    output = model(input_tensor)
    if class_idx is None:
        class_idx = torch.argmax(output, dim=1).item()

    # Backward pass
    model.zero_grad()
    loss = output[0, class_idx]
    loss.backward()

    grads = gradients[0].cpu().data.numpy()[0]
    fmap = target_layer(input_tensor).cpu().data.numpy()[0]

    # Grad-CAM calculation
    weights = np.mean(grads, axis=(1,2))
    cam = np.zeros(fmap.shape[1:], dtype=np.float32)
    for i, w in enumerate(weights):
        cam += w * fmap[i]
    cam = np.maximum(cam, 0)
    cam = cam / cam.max()
    cam = np.uint8(cam * 255)
    cam = Image.fromarray(cam).resize(input_image.size, Image.ANTIALIAS)
    return cam

# -------------------
# Prediction + Grad-CAM
# -------------------
def predict_with_gradcam(img):
    # Predict
    input_tensor = transform(img).unsqueeze(0).to(device)
    output = model(input_tensor)
    pred_idx = torch.argmax(output, dim=1).item()

    # Map class index to label (adjust this list to your dataset)
    labels = ['Normal', 'Diabetic Retinopathy', 'Glaucoma', 'Cataract', 'Other']
    pred_label = labels[pred_idx]

    # Grad-CAM overlay
    cam = grad_cam(img, class_idx=pred_idx)
    img_with_cam = Image.blend(img.convert("RGB"), cam.convert("RGB"), alpha=0.5)

    return pred_label, img_with_cam

# -------------------
# Launch Gradio app
# -------------------
interface = gr.Interface(
    fn=predict_with_gradcam,
    inputs=gr.Image(type="pil"),
    outputs=[gr.Textbox(label="Predicted Disease"), gr.Image(label="Grad-CAM Heatmap")],
    title="Retinal Disease Detection",
    description="Upload a fundus image to get predicted disease and Grad-CAM heatmap."
)

interface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f6c32cdd5511314805.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [35]:
!pip install --upgrade gradio --quiet

import torch
from torchvision import models, transforms
from PIL import Image
import gradio as gr
import numpy as np
import os

# -------------------
# Settings
# -------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
num_classes = 5
model_path = "/content/drive/MyDrive/retinalnewproject/results/APTOS_efficientnet_phase1.pth"

# -------------------
# Preprocessing
# -------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# -------------------
# Load model
# -------------------
model = models.efficientnet_b0(weights=None, num_classes=num_classes)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()
model.to(device)

# -------------------
# Grad-CAM function
# -------------------
def grad_cam(input_image, class_idx=None):
    gradients = []

    def save_gradients(module, grad_input, grad_output):
        gradients.append(grad_output[0])

    target_layer = model.features[-1]
    target_layer.register_full_backward_hook(save_gradients)

    input_tensor = transform(input_image).unsqueeze(0).to(device)
    input_tensor.requires_grad = True
    output = model(input_tensor)
    if class_idx is None:
        class_idx = torch.argmax(output, dim=1).item()
    model.zero_grad()
    loss = output[0, class_idx]
    loss.backward()

    grads = gradients[0].cpu().data.numpy()[0]
    fmap = target_layer(input_tensor).cpu().data.numpy()[0]
    weights = np.mean(grads, axis=(1, 2))
    cam = np.zeros(fmap.shape[1:], dtype=np.float32)
    for i, w in enumerate(weights):
        cam += w * fmap[i]
    cam = np.maximum(cam, 0)
    cam = cam / (cam.max() + 1e-8)
    cam = np.uint8(cam * 255)
    cam = Image.fromarray(cam).resize(input_image.size, Image.ANTIALIAS).convert("RGB")
    return cam

# -------------------
# Prediction + Grad-CAM
# -------------------
labels = ['Normal', 'Diabetic Retinopathy', 'Glaucoma', 'Cataract', 'Other']

def predict_multiple(file_paths):
    preds = []
    cams = []
    if not isinstance(file_paths, list):
        file_paths = [file_paths]

    for f in file_paths:
        img = Image.open(f).convert("RGB")
        input_tensor = transform(img).unsqueeze(0).to(device)
        output = model(input_tensor)
        pred_idx = torch.argmax(output, dim=1).item()
        pred_label = labels[pred_idx]
        preds.append(f"{os.path.basename(f)}: {pred_label}")

        cam = grad_cam(img, class_idx=pred_idx)
        blended = Image.blend(img, cam, alpha=0.5)
        cams.append(blended)

    # Return predictions as a single string
    return "\n".join(preds), cams

# -------------------
# Gradio Interface
# -------------------
interface = gr.Interface(
    fn=predict_multiple,
    inputs=gr.File(file_types=[".png", ".jpg", ".jpeg"], file_count="multiple", label="Upload Fundus Images"),
    outputs=[gr.Textbox(label="Predicted Diseases"), gr.Gallery(label="Grad-CAM Heatmaps")],
    title="Retinal Disease Detection",
    description="Upload one or more fundus images to get predictions and Grad-CAM heatmaps."
)

interface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5799619c6f7b159e42.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [20]:
import os

# Base dataset folder
dataset_base = "/content/drive/MyDrive/retinalnewproject/datasets"

# List all datasets
for dataset_name in os.listdir(dataset_base):
    dataset_path = os.path.join(dataset_base, dataset_name)
    if os.path.isdir(dataset_path):
        print(f"\nDataset: {dataset_name}")

        # List CSV files
        csv_files = [f for f in os.listdir(dataset_path) if f.endswith(".csv")]
        print("  CSV files:")
        for f in csv_files:
            print(f"    {os.path.join(dataset_path, f)}")

        # List image folders and first few images
        img_folders = [f for f in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, f))]
        print("  Image folders:")
        for f in img_folders:
            img_folder_path = os.path.join(dataset_path, f)
            imgs = os.listdir(img_folder_path)
            print(f"    {img_folder_path} ({len(imgs)} files)")
            print(f"      Sample images: {imgs[:5]}")



Dataset: DRIVE
  CSV files:
  Image folders:
    /content/drive/MyDrive/retinalnewproject/datasets/DRIVE/DRIVE (2 files)
      Sample images: ['training', 'test']

Dataset: .ipynb_checkpoints
  CSV files:
  Image folders:

Dataset: APTOS
  CSV files:
    /content/drive/MyDrive/retinalnewproject/datasets/APTOS/test.csv
    /content/drive/MyDrive/retinalnewproject/datasets/APTOS/train_1.csv
    /content/drive/MyDrive/retinalnewproject/datasets/APTOS/valid.csv
  Image folders:
    /content/drive/MyDrive/retinalnewproject/datasets/APTOS/val_images (1 files)
      Sample images: ['val_images']
    /content/drive/MyDrive/retinalnewproject/datasets/APTOS/train_images (1 files)
      Sample images: ['train_images']
    /content/drive/MyDrive/retinalnewproject/datasets/APTOS/test_images (1 files)
      Sample images: ['test_images']

Dataset: CHASEDB1
  CSV files:
  Image folders:
    /content/drive/MyDrive/retinalnewproject/datasets/CHASEDB1/Images (28 files)
      Sample images: ['Image_12L.

In [21]:
def get_actual_image_folder(base_folder):
    subfolders = [f for f in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, f))]
    if len(subfolders) == 0:
        raise ValueError(f"No subfolders found in {base_folder}")
    return os.path.join(base_folder, subfolders[0])

train_dir = get_actual_image_folder("/content/drive/MyDrive/retinalnewproject/datasets/APTOS/train_images")
val_dir   = get_actual_image_folder("/content/drive/MyDrive/retinalnewproject/datasets/APTOS/val_images")
test_dir  = get_actual_image_folder("/content/drive/MyDrive/retinalnewproject/datasets/APTOS/test_images")
